# Import necessary packages

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from random import sample

import re

from sklearn.metrics import mean_absolute_error, mean_squared_error

from timeit import default_timer as timer

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import original data and preprocessed data

In [4]:
sales_train_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/sales_train_validation.csv')
calendar = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/calendar.csv')

### Note: This Step is for the extraction of California State and Foods category 

In [5]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'HOBBIES')]

In [6]:
sales_train_val

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9707,HOBBIES_2_145_CA_4_validation,HOBBIES_2_145,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9708,HOBBIES_2_146_CA_4_validation,HOBBIES_2_146,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9709,HOBBIES_2_147_CA_4_validation,HOBBIES_2_147,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
9710,HOBBIES_2_148_CA_4_validation,HOBBIES_2_148,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
INPUT_DIR_2 = r'/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/List of Product ID according to 4 demand patterns/California/Hobbies'

list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

In [ ]:
len(sales_intermittent)

1301

In [ ]:
len(sales_lumpy)

395

In [ ]:
len(sales_erratic)

458

In [ ]:
len(sales_smooth)

3594

### Check by looking at column "id" to verify if we choose the right product (FOODS or HOBBIES or HOUSEHOLD)

In [8]:
sales_intermittent

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
10,HOBBIES_1_011_CA_1_validation,HOBBIES_1_011,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
17,HOBBIES_1_018_CA_1_validation,HOBBIES_1_018,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25,HOBBIES_1_026_CA_1_validation,HOBBIES_1_026,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
34,HOBBIES_1_035_CA_1_validation,HOBBIES_1_035,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
38,HOBBIES_1_039_CA_1_validation,HOBBIES_1_039,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9702,HOBBIES_2_140_CA_4_validation,HOBBIES_2_140,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9705,HOBBIES_2_143_CA_4_validation,HOBBIES_2_143,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9706,HOBBIES_2_144_CA_4_validation,HOBBIES_2_144,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9707,HOBBIES_2_145_CA_4_validation,HOBBIES_2_145,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# User-defined functions to calculate Metrics and Croston algorithm

In [9]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)

In [10]:
def Croston(ts,extra_periods=1,alpha=0.4):
    d = np.array(ts) # Transform the input into a numpy array
    cols = len(d) # Historical period length
    d = np.append(d,[np.nan]*extra_periods) # Append np.nan into the demand array to cover future periods
    
    #level (a), periodicity(p) and forecast (f)
    a,p,f = np.full((3,cols+extra_periods),np.nan)
    q = 1 #periods since last demand observation
    
    # Initialization
    first_occurence = np.argmax(d[:cols]>0)
    a[0] = d[first_occurence]
    p[0] = 1 + first_occurence
    f[0] = a[0]/p[0]
    # Create all the t+1 forecasts
    for t in range(0,cols):        
        if d[t] > 0:
            a[t+1] = alpha*d[t] + (1-alpha)*a[t] 
            p[t+1] = alpha*q + (1-alpha)*p[t]
            f[t+1] = a[t+1]/p[t+1]
            q = 1           
        else:
            a[t+1] = a[t]
            p[t+1] = p[t]
            f[t+1] = f[t]
            q += 1
       
    # Future Forecast 
    a[cols+1:cols+extra_periods] = a[cols]
    p[cols+1:cols+extra_periods] = p[cols]
    f[cols+1:cols+extra_periods] = f[cols]
                      
    df = pd.DataFrame.from_dict({"Demand":d,"Forecast":f,"Period":p,"Level":a,"Error":d-f})
    return df

# Format sales data and then merge with calendar data

In [11]:
sales_pattern = sales_intermittent.copy()

In [12]:
sales_pattern_py = sales_pattern.copy()

list_pattern_py = sales_pattern_py.id.unique().tolist()
list_pattern_py = [(re.sub(r'_validation', r"", x)) for x in list_pattern_py]

sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
df_pattern.drop('Date', axis=1, inplace=True)
df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
df_pattern.index.names = ['Date']
df_pattern = df_pattern.astype('float64')

df_pattern.iloc[:, 1:].columns = df_pattern.iloc[:, 1:].columns.str.rstrip('_validation')
df_pattern.columns = df_pattern.columns.str.replace(r'_validation', '')
df_pattern.reset_index(inplace=True)

calendar['Date'] = pd.to_datetime(calendar['date'])

dataframe = pd.merge(df_pattern,
                     calendar[['wm_yr_wk', 'Date']],
                     on = 'Date')

not_enough_seven_days = dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})[dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})['Date'] < 7].index.tolist()
dataframe = dataframe[~dataframe['wm_yr_wk'].isin(not_enough_seven_days)]

dataframe.set_index('wm_yr_wk', inplace=True)
dataframe.drop('Date', axis=1, inplace=True)

dataframe_weekly = dataframe.groupby('wm_yr_wk').sum().reset_index()

# Set value for parameters

In [13]:
list_params = [round(item, 2) for item in list(np.arange(0.05, 0.95, 0.05))]

In [14]:
list_params

[0.05,
 0.1,
 0.15,
 0.2,
 0.25,
 0.3,
 0.35,
 0.4,
 0.45,
 0.5,
 0.55,
 0.6,
 0.65,
 0.7,
 0.75,
 0.8,
 0.85,
 0.9]

In [15]:
start_train_week = 11318
end_train_week = 11610
n_pred_weeks = 2
num_train_needed = 50
validation_training_ratio = 0.95
num_id_sample_for_tuning = 200

In [16]:
sample_list = sample(list_pattern_py, num_id_sample_for_tuning)

In [17]:
dataframe_check = dataframe_weekly[sample_list].iloc[-2:].T

dataframe_check['Check'] = np.where((dataframe_check[271] == 0) &
                                    (dataframe_check[272] == 0), 1, 0)

sample_removed = dataframe_check[dataframe_check['Check'] == 1].index.tolist()

In [18]:
sample_list_filtered = [x for x in sample_list if (x not in sample_removed)]

In [19]:
len(sample_list_filtered)

106

# Hyperparameter tuning for Croston

In [20]:
train_data = dataframe_weekly[(dataframe_weekly['wm_yr_wk'] >= start_train_week) & (dataframe_weekly['wm_yr_wk'] <= end_train_week)]
test_data = dataframe_weekly.iloc[n_pred_weeks*-1:]

train_data_first_part = train_data.iloc[:int(len(train_data)*validation_training_ratio), :]
train_data_second_part = train_data.iloc[int(len(train_data)*validation_training_ratio):, :]

In [21]:
start = timer()

df_result_all_params = pd.DataFrame()
for product in sample_list_filtered:
    print('Currently Running: %s' % product)
    print('Progressing: {0} %'.format(round(sample_list_filtered.index(product) / len(sample_list_filtered) * 100, 2)))

    train_first = train_data_first_part[product]
    train_second = train_data_second_part[product]
    for i in list_params:
        predictions = list()
        history = [x for x in train_first[num_train_needed*-1:]]
        for t in train_second.index.tolist():
            yhat = Croston(history, extra_periods=1, alpha=i)['Forecast'].iloc[-1]
            predictions.append(yhat)
            history.append(train_second[t])
        df_result_temp = pd.DataFrame({'Product': [product for count in range(len(train_second))],
                                        'Actual Data': train_second,
                                        'Forecast': predictions,
                                        'Alpha': [i for count in range(len(train_second))]})
        df_result_all_params = df_result_all_params.append(df_result_temp, ignore_index=True)

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

Currently Running: HOBBIES_1_117_CA_4
Progressing: 0.0 %
Currently Running: HOBBIES_1_062_CA_2
Progressing: 0.94 %
Currently Running: HOBBIES_1_325_CA_3
Progressing: 1.89 %
Currently Running: HOBBIES_2_099_CA_3
Progressing: 2.83 %
Currently Running: HOBBIES_1_305_CA_3
Progressing: 3.77 %
Currently Running: HOBBIES_1_406_CA_4
Progressing: 4.72 %
Currently Running: HOBBIES_1_273_CA_4
Progressing: 5.66 %
Currently Running: HOBBIES_2_087_CA_2
Progressing: 6.6 %
Currently Running: HOBBIES_2_105_CA_3
Progressing: 7.55 %
Currently Running: HOBBIES_1_333_CA_1
Progressing: 8.49 %
Currently Running: HOBBIES_1_294_CA_1
Progressing: 9.43 %
Currently Running: HOBBIES_2_052_CA_2
Progressing: 10.38 %
Currently Running: HOBBIES_1_192_CA_1
Progressing: 11.32 %
Currently Running: HOBBIES_1_181_CA_3
Progressing: 12.26 %
Currently Running: HOBBIES_1_170_CA_3
Progressing: 13.21 %
Currently Running: HOBBIES_1_083_CA_1
Progressing: 14.15 %
Currently Running: HOBBIES_2_104_CA_2
Progressing: 15.09 %
Currently 

In [22]:
df_result_all_params

,Product,Actual Data,Forecast,Alpha
0,HOBBIES_1_117_CA_4,0.0,1.095953,0.05
1,HOBBIES_1_117_CA_4,1.0,1.095953,0.05
2,HOBBIES_1_117_CA_4,0.0,1.019011,0.05
3,HOBBIES_1_117_CA_4,1.0,1.019011,0.05
4,HOBBIES_1_117_CA_4,1.0,0.984694,0.05
...,...,...,...,...
15259,HOBBIES_2_006_CA_2,0.0,0.355530,0.90
15260,HOBBIES_2_006_CA_2,2.0,0.355530,0.90
15261,HOBBIES_2_006_CA_2,1.0,0.912904,0.90
15262,HOBBIES_2_006_CA_2,0.0,0.983642,0.90


# Summay all metrics

In [23]:
def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

In [24]:
start = timer()

df_result_metrics_all_params = df_result_all_params.groupby(['Product', 'Alpha']).apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.07293498131666686 minutes


In [25]:
df_result_metrics_all_params

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOBBIES_1_002_CA_3,0.05,0.580698,52.1443,56.3541,45.8509,0.912526,1.183707
1,HOBBIES_1_002_CA_3,0.10,0.525417,47.1803,52.9463,38.4618,0.825656,1.159003
2,HOBBIES_1_002_CA_3,0.15,0.539102,48.4092,54.2226,37.9684,0.847161,1.166950
3,HOBBIES_1_002_CA_3,0.20,0.555825,49.9108,55.7205,37.8273,0.873439,1.185540
4,HOBBIES_1_002_CA_3,0.25,0.575203,51.6509,57.3942,38.0442,0.903890,1.213266
...,...,...,...,...,...,...,...,...
1903,HOBBIES_2_146_CA_3,0.70,0.944018,67.4299,75.2598,104.3581,2.022896,2.302088
1904,HOBBIES_2_146_CA_3,0.75,0.957869,68.4192,76.1917,105.6128,2.052577,2.358247
1905,HOBBIES_2_146_CA_3,0.80,0.976319,69.7371,77.4489,107.0257,2.092112,2.414937
1906,HOBBIES_2_146_CA_3,0.85,0.994696,71.0497,78.6863,108.2347,2.131492,2.471894


# Check what products has unexpected metrics result

In [26]:
df_result_metrics_all_params[df_result_metrics_all_params['MASE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
1710,HOBBIES_2_103_CA_4,0.05,inf,inf,200.0,NaN,0.827710,0.827710
1711,HOBBIES_2_103_CA_4,0.10,inf,inf,200.0,NaN,0.706604,0.706604
1712,HOBBIES_2_103_CA_4,0.15,inf,inf,200.0,NaN,0.614431,0.614431
1713,HOBBIES_2_103_CA_4,0.20,inf,inf,200.0,NaN,0.543595,0.543595
1714,HOBBIES_2_103_CA_4,0.25,inf,inf,200.0,NaN,0.487315,0.487315
1715,HOBBIES_2_103_CA_4,0.30,inf,inf,200.0,NaN,0.440939,0.440939
1716,HOBBIES_2_103_CA_4,0.35,inf,inf,200.0,NaN,0.401555,0.401555
1717,HOBBIES_2_103_CA_4,0.40,inf,inf,200.0,NaN,0.367303,0.367303
1718,HOBBIES_2_103_CA_4,0.45,inf,inf,200.0,NaN,0.336916,0.336916
1719,HOBBIES_2_103_CA_4,0.50,inf,inf,200.0,NaN,0.309477,0.309477


In [27]:
df_result_metrics_all_params[df_result_metrics_all_params['WMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
1710,HOBBIES_2_103_CA_4,0.05,inf,inf,200.0,NaN,0.827710,0.827710
1711,HOBBIES_2_103_CA_4,0.10,inf,inf,200.0,NaN,0.706604,0.706604
1712,HOBBIES_2_103_CA_4,0.15,inf,inf,200.0,NaN,0.614431,0.614431
1713,HOBBIES_2_103_CA_4,0.20,inf,inf,200.0,NaN,0.543595,0.543595
1714,HOBBIES_2_103_CA_4,0.25,inf,inf,200.0,NaN,0.487315,0.487315
1715,HOBBIES_2_103_CA_4,0.30,inf,inf,200.0,NaN,0.440939,0.440939
1716,HOBBIES_2_103_CA_4,0.35,inf,inf,200.0,NaN,0.401555,0.401555
1717,HOBBIES_2_103_CA_4,0.40,inf,inf,200.0,NaN,0.367303,0.367303
1718,HOBBIES_2_103_CA_4,0.45,inf,inf,200.0,NaN,0.336916,0.336916
1719,HOBBIES_2_103_CA_4,0.50,inf,inf,200.0,NaN,0.309477,0.309477


In [28]:
df_result_metrics_all_params[df_result_metrics_all_params['SMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
1080,HOBBIES_1_376_CA_2,0.05,1.705093,97.4339,NaN,96.9207,0.730754,0.980943
1081,HOBBIES_1_376_CA_2,0.10,1.703078,97.3188,NaN,96.7825,0.729891,0.980099
1082,HOBBIES_1_376_CA_2,0.15,1.700661,97.1806,NaN,96.6168,0.728855,0.979088
1083,HOBBIES_1_376_CA_2,0.20,1.697729,97.0131,NaN,96.4157,0.727598,0.977865
1084,HOBBIES_1_376_CA_2,0.25,1.694128,96.8073,NaN,96.1688,0.726055,0.976369
1085,HOBBIES_1_376_CA_2,0.30,1.689646,96.5512,NaN,95.8614,0.724134,0.974517
1086,HOBBIES_1_376_CA_2,0.35,1.683986,96.2278,NaN,95.4733,0.721708,0.972194
1087,HOBBIES_1_376_CA_2,0.40,1.676724,95.8128,NaN,94.9753,0.718596,0.969240
1088,HOBBIES_1_376_CA_2,0.45,1.667249,95.2713,NaN,94.3256,0.714535,0.965436
1089,HOBBIES_1_376_CA_2,0.50,1.654671,94.5526,NaN,93.4631,0.709145,0.960477


In [29]:
df_result_metrics_all_params[df_result_metrics_all_params['MAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
1710,HOBBIES_2_103_CA_4,0.05,inf,inf,200.0,NaN,0.827710,0.827710
1711,HOBBIES_2_103_CA_4,0.10,inf,inf,200.0,NaN,0.706604,0.706604
1712,HOBBIES_2_103_CA_4,0.15,inf,inf,200.0,NaN,0.614431,0.614431
1713,HOBBIES_2_103_CA_4,0.20,inf,inf,200.0,NaN,0.543595,0.543595
1714,HOBBIES_2_103_CA_4,0.25,inf,inf,200.0,NaN,0.487315,0.487315
1715,HOBBIES_2_103_CA_4,0.30,inf,inf,200.0,NaN,0.440939,0.440939
1716,HOBBIES_2_103_CA_4,0.35,inf,inf,200.0,NaN,0.401555,0.401555
1717,HOBBIES_2_103_CA_4,0.40,inf,inf,200.0,NaN,0.367303,0.367303
1718,HOBBIES_2_103_CA_4,0.45,inf,inf,200.0,NaN,0.336916,0.336916
1719,HOBBIES_2_103_CA_4,0.50,inf,inf,200.0,NaN,0.309477,0.309477


In [30]:
df_result_metrics_all_params[df_result_metrics_all_params['RMSE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [31]:
df_result_metrics_all_params[df_result_metrics_all_params['MAE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


# Metrics Statistics

In [32]:
list_metrics = ['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE']

In [33]:
for mts in list_metrics:
    print('Percentage of unexpected values of', mts, 'is: {}'.format(df_result_metrics_all_params[df_result_metrics_all_params[mts].isin([np.nan, np.inf, -np.inf])].Product.nunique() / df_result_metrics_all_params.Product.nunique() * 100), "%")

Percentage of unexpected values of MASE is: 0.9433962264150944 %
Percentage of unexpected values of WMAPE is: 0.9433962264150944 %
Percentage of unexpected values of SMAPE is: 0.9433962264150944 %
Percentage of unexpected values of MAPE is: 0.9433962264150944 %
Percentage of unexpected values of MAE is: 0.0 %
Percentage of unexpected values of RMSE is: 0.0 %


### Filter all rows that have unexpected metrics values

In [34]:
df_result_metrics_all_params = df_result_metrics_all_params[~df_result_metrics_all_params.isin([np.nan, np.inf, -np.inf]).any(1)]

### Get MEAN metrics value of each Alpha

In [35]:
df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()

,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
Alpha,,,,,,
0.05,1.052872,103.079359,122.271058,58.345138,1.394316,1.689810
0.10,1.019858,99.103972,118.322427,57.381748,1.326877,1.623727
0.15,0.996529,96.718612,114.393065,55.516362,1.269387,1.558904
0.20,0.984757,95.688133,111.311573,54.312275,1.234728,1.514461
0.25,0.981232,95.542694,108.992527,53.522568,1.214284,1.491460
0.30,0.980923,95.826419,107.261662,52.762533,1.202322,1.482852
0.35,0.984763,96.540785,106.263470,52.441661,1.197602,1.483370
0.40,0.993007,97.589172,105.901228,52.646623,1.203204,1.489719
0.45,1.002495,98.720970,105.890232,53.087754,1.214101,1.499739


In [36]:
for mts in list_metrics:
    print('The optimum Alpha based on Mean', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()[mts].idxmin()))

The optimum Alpha based on Mean MASE is: 0.3
The optimum Alpha based on Mean WMAPE is: 0.25
The optimum Alpha based on Mean SMAPE is: 0.45
The optimum Alpha based on Mean MAPE is: 0.35
The optimum Alpha based on Mean MAE is: 0.35
The optimum Alpha based on Mean RMSE is: 0.3


### Get MEDIAN metrics value of each Alpha

In [ ]:
df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()

,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
Alpha,,,,,,
0.05,1.309016,86.8895,115.6938,71.9423,3.954124,4.607941
0.10,1.111668,75.8770,98.6324,63.9110,3.670187,4.330000
0.15,0.997008,76.1758,99.0938,59.9314,3.589330,4.243416
0.20,0.892849,74.4309,98.0758,59.4701,3.429005,4.185133
0.25,0.847639,74.6550,83.4834,60.6409,3.447667,4.177697
0.30,0.855515,74.9120,82.6296,61.3418,3.566326,4.108989
0.35,0.865497,73.7833,82.8383,62.4621,3.548911,4.066846
0.40,0.874011,73.3134,83.3269,65.0429,3.529703,4.042035
0.45,0.894779,72.5240,83.6190,65.5492,3.506639,4.033434


In [ ]:
for mts in list_metrics:
    print('The optimum Alpha based on Median of', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()[mts].idxmin()))

The optimum Alpha based on Median of MASE is: 0.25
The optimum Alpha based on Median of WMAPE is: 0.75
The optimum Alpha based on Median of SMAPE is: 0.9
The optimum Alpha based on Median of MAPE is: 0.2
The optimum Alpha based on Median of MAE is: 0.85
The optimum Alpha based on Median of RMSE is: 0.45


# Run Croston model after decide best paramaters

In [37]:
best_alpha = 0.35

In [38]:
dataframe_weekly

,wm_yr_wk,HOBBIES_1_002_CA_3,HOBBIES_1_002_CA_4,HOBBIES_1_003_CA_2,HOBBIES_1_003_CA_3,HOBBIES_1_003_CA_4,HOBBIES_1_005_CA_4,HOBBIES_1_006_CA_4,HOBBIES_1_007_CA_2,HOBBIES_1_011_CA_1,...,HOBBIES_2_141_CA_2,HOBBIES_2_143_CA_4,HOBBIES_2_144_CA_4,HOBBIES_2_145_CA_1,HOBBIES_2_145_CA_4,HOBBIES_2_146_CA_1,HOBBIES_2_146_CA_2,HOBBIES_2_146_CA_3,HOBBIES_2_146_CA_4,HOBBIES_2_147_CA_1
0,11101,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0
1,11102,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0
2,11103,3.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,...,0.0,0.0,0.0,0.0,0.0,2.0,3.0,2.0,1.0,1.0
3,11104,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0
4,11105,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,3.0,2.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,11608,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,3.0,0.0,0.0,0.0,0.0,2.0,2.0,5.0,0.0,0.0
269,11609,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,1.0,0.0,0.0,0.0,0.0,2.0,5.0,1.0,0.0,2.0
270,11610,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,0.0,2.0
271,11611,1.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,2.0,1.0,4.0,0.0,0.0


In [39]:
train_data = dataframe_weekly[(dataframe_weekly.wm_yr_wk >= start_train_week) & (dataframe_weekly.wm_yr_wk <= end_train_week)]
test_data = dataframe_weekly.iloc[n_pred_weeks*-1:]

In [40]:
df_result_best_params = pd.DataFrame()
for product in list_pattern_py:
    train_product = train_data[product]
    test_product = test_data[product]
    predictions = list()
    history = [x for x in train_product[num_train_needed*-1:]]
    for t in test_product.index.tolist():
        yhat = Croston(history, extra_periods=1, alpha=best_alpha)['Forecast'].iloc[-1]
        predictions.append(yhat)
        history.append(test_product[t])
    df_result_temp = pd.DataFrame({'Product': [product for count in range(len(test_product))],
                                    'Actual Data': test_product,
                                    'Forecast': predictions})
    df_result_best_params = df_result_best_params.append(df_result_temp, ignore_index=True)

In [41]:
df_result_best_params

,Product,Actual Data,Forecast
0,HOBBIES_1_011_CA_1,1.0,0.381216
1,HOBBIES_1_011_CA_1,2.0,0.468050
2,HOBBIES_1_018_CA_1,0.0,0.475771
3,HOBBIES_1_018_CA_1,0.0,0.475771
4,HOBBIES_1_026_CA_1,0.0,0.000000
...,...,...,...
1733,HOBBIES_2_144_CA_4,0.0,0.142758
1734,HOBBIES_2_145_CA_4,0.0,1.136153
1735,HOBBIES_2_145_CA_4,0.0,1.136153
1736,HOBBIES_2_146_CA_4,0.0,0.472163


In [42]:
df_result_final = df_result_best_params.groupby('Product').apply(get_metrics_result_all_params).reset_index()

In [43]:
df_result_final

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOBBIES_1_002_CA_3,inf,38.9135,32.4414,38.9135,0.389135,0.394959
1,HOBBIES_1_002_CA_4,0.565067,113.0133,133.5996,50.2989,1.130133,1.136933
2,HOBBIES_1_003_CA_2,NaN,NaN,NaN,NaN,0.000000,0.000000
3,HOBBIES_1_003_CA_3,NaN,NaN,NaN,NaN,0.000000,0.000000
4,HOBBIES_1_003_CA_4,NaN,NaN,NaN,NaN,0.000000,0.000000
...,...,...,...,...,...,...,...
864,HOBBIES_2_146_CA_1,0.462889,92.5778,119.9758,33.2997,0.925778,0.961537
865,HOBBIES_2_146_CA_2,0.525280,87.5467,111.9687,113.4409,5.252801,6.487696
866,HOBBIES_2_146_CA_3,0.603115,17.2318,17.2792,17.6306,0.603115,0.603190
867,HOBBIES_2_146_CA_4,inf,inf,200.0000,NaN,0.472163,0.472163


In [44]:
df_result_final.isnull().sum()

Product      0
MASE       132
WMAPE      132
SMAPE      132
MAPE       393
MAE          0
RMSE         0
dtype: int64

In [45]:
df_result_final.MAPE.mean()

57.54728382352941

In [46]:
df_result_final.to_csv('Croston_Intermittent_Metrics.csv')